#### Notebook setup

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import time
import warnings
import os, sys
from types import SimpleNamespace

_lib_path, _abs_path = None, os.path.abspath('.')
if os.path.dirname(_abs_path) == 'multirotor':
    _lib_path = os.path.abspath(os.path.join(_abs_path, '..'))
elif os.path.isdir('multirotor'):
    _lib_path = _abs_path
if _lib_path is not None and _lib_path not in sys.path:
    sys.path.append(_lib_path)

import matplotlib.pyplot as plt
import gym
import numpy as np
from tqdm.auto import tqdm, trange
try:
    import pygazebo
    from pygazebo.msg.v11.world_control_pb2 import WorldControl
    from gym_env import RemoteOctorotor
    from gazebo_comms import GazeboCommms
    from msgs.messages_pb2 import Action, State
except ImportError:
    warnings.warn('Could not import pygazebo. Remote communication with gazebo back-end will not work.')

from multirotor.helpers import control_allocation_matrix
from multirotor.vehicle import MotorParams, VehicleParams, PropellerParams, SimulationParams
from multirotor.control import PosController, AttController, AltController, Controller
from multirotor.simulation import Multirotor, Propeller, Motor, Battery
from multirotor.visualize import VehicleDrawing
from multirotor.coords import body_to_inertial, inertial_to_body, direction_cosine_matrix, angular_to_euler_rate
from multirotor.env import DynamicsMultirotorEnv as LocalOctorotor
from multirotor.trajectories import Trajectory

/home/zhanga/miniconda3/envs/gazebo/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Plotting parameters
SMALL_SIZE = 16
MEDIUM_SIZE = 16
BIGGER_SIZE = 20

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE, titlesize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title


### Parameters

In [3]:
# Tarot T18 params
mp = MotorParams(
    moment_of_inertia=5e-5,
    resistance=0.27,
    k_emf=0.0265
)
pp = PropellerParams(
    moment_of_inertia=1.86e-6,
    use_thrust_constant=True,
    k_thrust=9.8419e-05,
    k_drag=1.8503e-06,
    motor=mp
)
vp = VehicleParams(
    propellers=[pp] * 8,
    angles=np.linspace(0, -2*np.pi, num=8, endpoint=False) + 0.375 * np.pi,
    distances=np.ones(8) * 0.635,
    clockwise=[-1,1,-1,1,-1,1,-1,1],
    mass=10.66,
    inertia_matrix=np.asarray([
        [0.2206, 0, 0],
        [0, 0.2206, 0.],
        [0, 0, 0.4238]
    ])
)
sp = SimulationParams(0.0025, 9.81)

### Message Passing

In [4]:
gc = GazeboCommms()
# List all topics gazebo is publishing to:
gc.manager.publications()

[('/gazebo/server/control', 'gazebo.msgs.ServerControl'),
 ('/gazebo/test/atmosphere', 'gazebo.msgs.Atmosphere'),
 ('/gazebo/test/playback_control', 'gazebo.msgs.LogPlaybackControl'),
 ('/gazebo/test/undo_redo', 'gazebo.msgs.UndoRedo'),
 ('/gazebo/test/scene', 'gazebo.msgs.Scene'),
 ('/gazebo/test/request', 'gazebo.msgs.Request'),
 ('/gazebo/test/visual', 'gazebo.msgs.Visual'),
 ('/gazebo/test/physics/contacts', 'gazebo.msgs.Contacts'),
 ('/gazebo/test/world_stats', 'gazebo.msgs.WorldStatistics'),
 ('/tarot/motors', 'tarotPB.msgs.Vector8d'),
 ('/tarot/action', 'tarotPB.msgs.Action'),
 ('/gazebo/test/user_camera/pose', 'gazebo.msgs.Pose'),
 ('/gazebo/test/log/control', 'gazebo.msgs.LogControl'),
 ('/gazebo/test/user_cmd', 'gazebo.msgs.UserCmd'),
 ('/gazebo/test/joint', 'gazebo.msgs.Joint'),
 ('/gazebo/test/wind', 'gazebo.msgs.Wind'),
 ('/gazebo/test/user_cmd_stats', 'gazebo.msgs.UserCmdStats'),
 ('/gazebo/test/light/modify', 'gazebo.msgs.Light'),
 ('/gazebo/test/world_control', 'gazebo.

In [5]:
# Create a protobuf message
a = Action()
a.type = a.DYNAMICS
a.values.extend([0., 0., 0., 0., 0., 0.])
# We can send the message over TCP. However, we will not get the state vector
# back in response, because the custom environment plugin is only listening on UDP.
# gc.send_TCP(a, '/tarot/action')

In [6]:
# Reset
a = Action()
a.reset = True
a.state.position.extend([1,0,1])
a.state.velocity.extend([3,0,0])
a.state.orientation.extend([0,0,0])
a.state.angular_rate.extend([0,0,0])
a
# gc.send_TCP(a, '/tarot/action')

reset: true
state {
  position: 1.0
  position: 0.0
  position: 1.0
  velocity: 3.0
  velocity: 0.0
  velocity: 0.0
  orientation: 0.0
  orientation: 0.0
  orientation: 0.0
  angular_rate: 0.0
  angular_rate: 0.0
  angular_rate: 0.0
}

In [7]:
# Send action over UDP. The gazebo plugin will respond back with the state vector when getting a UDP message
gc.send_UDP(a)

(None, Exception('Timeout reached waiting for response.'))

In [8]:
# Create and send a default gazebo message. These packets are directly handled
# by gazebo, and not the custom plugin
c = WorldControl()
c.reset.all = True
c.multi_step = 100
gc.send_TCP(c, '/gazebo/test/world_control')

### AP Simulation

In [9]:
import socket
import time
import json
import struct
from multirotor.coords import direction_cosine_matrix, body_to_inertial

In [10]:
def wind(t, m):
    w_inertial = np.asarray([5 * np.sin(t * 2 * np.pi / 4000), 0, 0])
    dcm = direction_cosine_matrix(*m.orientation)
    return inertial_to_body(w_inertial, dcm)

In [11]:
def changeToJSONString(curr_time, state, accel):
    phys_time = curr_time
    pos = state[0:3].tolist()
    velo = state[3:6]
    euler = state[6:9].tolist()
    gyro = state[9:12].tolist()
    accel = accel.tolist()

    pos[2] = -1*pos[2]
    pos[0], pos[1] = pos[1], pos[0]

    dcm = direction_cosine_matrix(*euler)
    v_inertial = body_to_inertial(velo, dcm).tolist()
    v_inertial[2] = -1*v_inertial[2]
    v_inertial[0], v_inertial[1] = v_inertial[1], v_inertial[0]

    gyro[0], gyro[1] = gyro[1], gyro[0]
    gyro[2] = -1*gyro[2]
    euler[0], euler[1] = euler[1], euler[0]
    euler[2] = -1*euler[2]

    # Build JSON format
    IMU_fmt = {
        "gyro" : gyro,
        "accel_body" : accel
    }
    JSON_fmt = {
        "timestamp" : phys_time,
        "imu" : IMU_fmt,
        "position" : pos,
        "attitude" : euler,
        "velocity" : v_inertial
    }

    JSON_string = "\n" + json.dumps(JSON_fmt,separators=(',', ':')) + "\n"

    return JSON_string


In [12]:
def ap_sim(env, sock, steps=600000, disturbance=None):
    command_logger = {}
    state_logger = {}
    
    disturb_force, disturb_torque = 0., 0

    curr_time = 0  # Get the current time in seconds
    RATE_HZ = 400 #should be 400
    TIME_STEP = 1/RATE_HZ
    last_frame = -1
    frame_count = 0
    frame_time = time.time()
    print_frame_count = 500

    state = np.zeros(12)
    
    try:
        for i in range(0, steps):
            try:   
                data, addr = sock.recvfrom(100)
                parse_format = 'HHI16H'
                magic = 18458
                if len(data) != struct.calcsize(parse_format):
                    print("got packet of len %u, expected %u" % (len(data), struct.calcsize(parse_format)))
                    continue
                unpacked_data = struct.unpack(parse_format, data)
                if magic != unpacked_data[0]:
                    print("Incorrect protocol magic %u should be %u" % (unpacked_data[0], magic))
                    continue

                frame_rate = unpacked_data[1]
                frame_count = unpacked_data[2]
                pwm = np.array(unpacked_data[3:11])

                string_action = " ".join(str(element) for element in pwm.tolist())

                if frame_count < last_frame:
                    print('Reset controller')
                elif frame_count == last_frame:
                    # print('Duplicate frame')
                    continue
                if frame_count != last_frame + 1 and last_frame != 0:
                    print("Missed %u frames" % (frame_count - last_frame))
                    continue
                last_frame = frame_count
                
                curr_time += TIME_STEP

                action = (pwm-1000)*0.575

                f, t = m.get_forces_torques(action, state)
                state, *_ = env.step(np.asarray([*f, *t]))

                # These functions don't work for some reason??
                # accel = env.vehicle.dxdt_dynamics(0, state, [*f, *t])[3:6]
                # accel = env.vehicle._dxdt[3:6]
                accel = np.array([0, 0, -9.8])
                if disturbance is not None:
                    accel[0] += disturb_force[0]
                    accel[1] += disturb_force[1]
                    accel[2] += disturb_force[2] #change accel z frame to body
                JSON_string = changeToJSONString(curr_time, state, accel)

                sock.sendto(bytes(JSON_string,"ascii"), addr)
                
                if action[0] != 0:
                    command_logger[i] = action.tolist()
                    state_logger[i] = state.tolist()

            except json.JSONDecodeError:
                print("Invalid JSON data received")
            except socket.timeout:
                continue
            except KeyboardInterrupt:
                sock.close()
            except Exception as e:
                if not isinstance(e, OSError):
                    raise e
                sock.close()
                break
    finally:
        sock.close()
        
    return command_logger, state_logger


In [13]:

UDP_IP = "127.0.0.1"  # Listen on all available interfaces
UDP_PORT = 9002

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.bind((UDP_IP, UDP_PORT))
sock.settimeout(0.1)  # Set a timeout value of 1 second on the socket object

m = Multirotor(vp, sp)
m.reset()
env = RemoteOctorotor(vehicle=m)
env.reset()
action_log, state_log = ap_sim(env, sock, steps=600000, disturbance=None)

sock.close()

Exception in callback Manager.handle_normal_read(<Future finis...onnectError()>)
handle: <Handle Manager.handle_normal_read(<Future finis...onnectError()>)>
Traceback (most recent call last):
  File "/home/zhanga/miniconda3/envs/gazebo/lib/python3.9/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "/home/zhanga/miniconda3/envs/gazebo/lib/python3.9/site-packages/pygazebo/pygazebo.py", line 644, in handle_normal_read
    data = future.result()
  File "/home/zhanga/miniconda3/envs/gazebo/lib/python3.9/asyncio/futures.py", line 201, in result
    raise self._exception
  File "/home/zhanga/miniconda3/envs/gazebo/lib/python3.9/site-packages/pygazebo/pygazebo.py", line 363, in handle_read
    data = future.result()
  File "/home/zhanga/miniconda3/envs/gazebo/lib/python3.9/asyncio/futures.py", line 201, in result
    raise self._exception
  File "/home/zhanga/miniconda3/envs/gazebo/lib/python3.9/site-packages/pygazebo/pygazebo.py", line 308, in han

### Logging

In [ ]:
def printLogMotor(log, lower=0, upper=700):
    element_data = {}

    first_key = next(iter(log))
    for index in range(len(log[first_key])):
        element_data[index] = [data[index] for data in log.values()]

    plt.figure(figsize=(15, 5))

    # Plot the data for each element
    for index, data in element_data.items():
        # Filter data based on condition (value > 2000)
        filtered_data = [value if value >= lower and value <= upper else None for value in data]
        plt.plot(list(log.keys()), filtered_data, label=f"Motor {index+1}")

    # Set plot labels and title
    plt.xlabel("Frames")
    plt.ylabel("Motor command")
    plt.title("Motor Commands over time")

    # Add legend
    plt.legend(loc='upper left', fontsize='small')

    # Display the plot
    plt.show()

In [ ]:
def printStateLog(log):
    num_motors = len(log[next(iter(log))])

    fig, axs = plt.subplots(4, 3, figsize=(15, 15))

    labels = ["x", "y", "z", "vx", "vy", "vz", "roll", "pitch", "yaw", "vroll", "vpitch", "vyaw"]

    # Loop through each motor and plot it in the corresponding subplot
    for motor_index, ax in enumerate(axs.flatten()):
        if motor_index < num_motors:
            # Filter and plot the data for the current motor
            filtered_data = [data[motor_index] for data in log.values()]
            ax.plot(list(log.keys()), filtered_data, label=labels[motor_index])

            # Set plot labels and title
            ax.set_xlabel("Frames")
            ax.set_ylabel(labels[motor_index])
            ax.set_title(f"{labels[motor_index]} over time")

            # Add legend
            ax.legend(loc='upper left', fontsize='small')

    # Adjust layout to avoid overlapping titles and labels
    plt.tight_layout()

    # Display the subplots
    plt.show()

In [ ]:
printStateLog(state_log)

In [ ]:
printLogMotor(action_log)